In [3]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from data import data

,support,itemsets,Abs_support
0,0.8,(Eggs),4.0
1,1.0,(Kidney Beans),5.0
2,0.6,(Milk),3.0
3,0.6,(Onion),3.0
4,0.6,(Yogurt),3.0
5,0.8,"(Eggs, Kidney Beans)",4.0
6,0.6,"(Eggs, Onion)",3.0
7,0.6,"(Milk, Kidney Beans)",3.0
8,0.6,"(Onion, Kidney Beans)",3.0
9,0.6,"(Yogurt, Kidney Beans)",3.0


In [5]:
books = data.loadAndClean('../../data/booksummaries/booksummaries.txt')

In [6]:
books = books[['bookGenre', 'plotSum', 'bookTitle']]

In [10]:
corpus = list(books.bookGenre.apply(lambda row: row))
corpus[:1]

[['roman_à_clef',
  'satire',
  'childrens_literature',
  'speculative_fiction',
  'fiction']]

In [12]:
N = len(corpus)

te = TransactionEncoder()
te_ary = te.fit(corpus).transform(corpus)
df = pd.DataFrame(te_ary, columns=te.columns_)
df.head(5)

,absurdist_fiction,adventure,adventure_novel,albino_bias,alien_invasion,alternate_history,american_gothic_fiction,anthology,anthropology,anti-nuclear,...,vampire_fiction,war_novel,western,western_fiction,whodunit,wuxia,young_adult_literature,youth,zombie,zombies_in_popular_culture
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,True,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False


In [13]:
frequent_itemsets = apriori(df, min_support=0.0001, use_colnames=True)
frequent_itemsets['Abs_support'] = frequent_itemsets.support * N

,support,itemsets,Abs_support
0,0.002258,(absurdist_fiction),29.0
1,0.001791,(adventure),23.0
2,0.025699,(adventure_novel),330.0
3,0.000156,(albino_bias),2.0
4,0.017600,(alternate_history),226.0
5,0.000467,(anthology),6.0
6,0.000312,(anthropology),4.0
7,0.004361,(apocalyptic_and_post-apocalyptic_fiction),56.0
8,0.005996,(autobiographical_novel),77.0
9,0.009657,(autobiography),124.0


In [54]:
frequent_itemsets['num_of_genres'] = frequent_itemsets.itemsets.apply(lambda row: len([i for i in range(len(row))]))

In [66]:
pd.set_option('display.max_colwidth', 150)
frequent_itemsets.sort_values(by='Abs_support', ascending=False)[frequent_itemsets.num_of_genres > 2].head(20)

/anaconda3/envs/AVC/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,support,itemsets,Abs_support,num_of_genres
2614,0.077876,"(science_fiction, fiction, speculative_fiction)",1000.0,3
2216,0.073748,"(fiction, speculative_fiction, fantasy)",947.0,3
2514,0.048049,"(mystery, suspense, fiction)",617.0,3
1727,0.043455,"(fiction, childrens_literature, speculative_fiction)",558.0,3
2335,0.043377,"(science_fiction, speculative_fiction, fantasy)",557.0,3
2212,0.039171,"(science_fiction, fiction, fantasy)",503.0,3
4094,0.036835,"(science_fiction, fiction, speculative_fiction, fantasy)",473.0,4
1683,0.036212,"(childrens_literature, speculative_fiction, fantasy)",465.0,3
1664,0.026556,"(childrens_literature, fiction, fantasy)",341.0,3
3485,0.024609,"(fiction, childrens_literature, speculative_fiction, fantasy)",316.0,4


Now we have a dataframe that we can see all the combinations and how often they occur. This will help us make the rules to classify our multi-genre labels into fewer genres. 